In [2]:
import os
os.environ["LABEL_STUDIO_LOCAL_FILES_DOCUMENT_ROOT"] = "/home/patel_zeel"

import sys
import requests
from os.path import join, exists, basename
from glob import glob
from label_studio_sdk import Client
from copy import deepcopy

import torch
import numpy as np
from ultralytics.utils.ops import xyxyxyxy2xywhr, xywhr2xyxyxyxy
from ultralytics.utils.metrics import probiou
from tqdm.notebook import tqdm

import matplotlib.pyplot as plt
import warnings

from joblib import Parallel, delayed

sys.path.append("/home/patel_zeel/kilns_neurips24/scripts")

In [3]:
from utils import yolo_obb_to_label_studio_json, label_map
label_map

{'FCBK': 0, 'Zigzag': 1}

## Label Studio Initialize

In [3]:
LABEL_STUDIO_URL = 'http://10.0.62.168:8080'
API_KEY = '182f2ac0975d5cb5661c4ddc1b4ddc34d21c1867'

ls = Client(url=LABEL_STUDIO_URL, api_key=API_KEY)

## Process the input data

In [4]:
pred_txt_path = "/home/patel_zeel/kilns_neurips24/gujarat/labels"
image_read_dir = "/home/patel_zeel/kilns_neurips24/gujarat/images"

ls_project_name = f"Gujarat_OBB_v2"

pred_files = sorted(glob(join(pred_txt_path, "*.txt")))
pred_base_names = set([basename(f) for f in pred_files])

print(len(pred_files))

3216


## Create Project

In [8]:
projects = ls.list_projects()
for project in projects:
    if project.title == ls_project_name:
        print("Project found. Using it...")
        break
else:
    print("No project found. Creating...")
    project = ls.start_project(title=ls_project_name, label_config='''
<View>
  <Image name="image" value="$image" zoom="true"/>
  <RectangleLabels name="label" toName="image">
    
    
  <Label value="Zigzag" background="#c0ff9e"/><Label value="FCBK" background="#D4380D"/></RectangleLabels>
</View>
    ''')
storages = project.get_import_storages()
if len(storages) == 0:
    print("No storage found. Creating...")
    project.connect_local_import_storage(image_read_dir, regex_filter=".*png")
else:
    assert len(storages) == 1
    print("Storage found. Using it...")
storage = storages[0]
if storage['last_sync'] is None:
    print("Syncing storage...")
    project.sync_storage(storage['type'], storage['id'])
else:
    print("Storage already synced.")

print("Getting tasks...")
tasks = project.get_tasks()
len(tasks)

Project found. Using it...
Storage found. Using it...
Syncing storage...
Getting tasks...


3216

In [9]:
tasks[0]

{'id': 194088,
 'predictions': [],
 'annotations': [],
 'drafts': [],
 'annotators': [],
 'inner_id': 1,
 'cancelled_annotations': 0,
 'total_annotations': 0,
 'total_predictions': 0,
 'completed_at': None,
 'annotations_results': '',
 'predictions_results': '',
 'predictions_score': None,
 'file_upload': None,
 'storage_filename': '/home/patel_zeel/kilns_neurips24/gujarat/images/20.20,73.07.png',
 'annotations_ids': '',
 'predictions_model_versions': '',
 'avg_lead_time': None,
 'draft_exists': False,
 'updated_by': [],
 'data': {'image': '/data/local-files/?d=home/patel_zeel/kilns_neurips24/gujarat/images/20.20%2C73.07.png'},
 'meta': {},
 'created_at': '2024-08-03T17:08:16.876210Z',
 'updated_at': '2024-08-03T17:08:16.876233Z',
 'is_labeled': False,
 'overlap': 1,
 'comment_count': 0,
 'unresolved_comment_count': 0,
 'last_comment_updated_at': None,
 'project': 65,
 'comment_authors': []}

In [12]:
## Delete all predictions before going ahead
os.system(f"curl -H 'Authorization: Token {API_KEY}' -X POST \"localhost:8080/api/dm/actions?id=delete_tasks_predictions&project={project.id}\"")

for task in tqdm(tasks):
    # part = tile.split("/")[-3]
    # name = tile.split("/")[-1].replace(".txt", ".png")
    # image_path = join(save_dir, name)
    image_path = task['storage_filename']
    txt_name = basename(image_path).replace(".png", ".txt")
    
    preds = np.loadtxt(join(pred_txt_path, txt_name))
    preds = np.atleast_2d(preds)

    results = []
    for pred in preds:
        label_id = int(pred[0])
        x1, y1, x2, y2, x3, y3, x4, y4 = pred[1:-1]
        score = pred[-1]
        # if score > score_threshold:
        prediction = yolo_obb_to_label_studio_json(label_id, x1, y1, x2, y2, x3, y3, x4, y4, score)
          # predictions.append(prediction)
        results.append(prediction['result'][0])
    if len(results) > 0:
      # I have tried parallelizing this but it doesn't work
      project.create_prediction(task['id'], results, model_version="v1")

{"processed_items":0,"detail":"Deleted 0 predictions"}

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    54  100    54    0     0   2250      0 --:--:-- --:--:-- --:--:--  2250


  0%|          | 0/3216 [00:00<?, ?it/s]

## Appendix

In [ ]:
# areas = []
# for item in json_list:
#     for pred in item["predictions"]:
#         area = pred['result'][0]['value']['width'] * pred['result'][0]['value']['height']
#         area = area * (1120 * 1120) / (100 * 100)
#         areas.append(area)
# len(areas)

In [ ]:
# max(areas), min(areas), np.mean(areas), np.median(areas)

In [ ]:
# import seaborn as sns

# sns.kdeplot(areas)

In [ ]:
# (areas < np.mean(areas) - 2 * np.std(areas)).sum()